# XSSClassifier:  XSS Attack Detection Approach Based on Machine Learning Classifier in Python

### Research/Related Work

Based on the white paper, XSSClassifier: An Efficient XSS Attack Detection Approach Based on Machine Learning Classifier on SNSs:


https://pdfs.semanticscholar.org/2c74/d8e94b73c35e8651189262d0c7f32e6cfc7c.pdf?_ga=2.60428845.1871262773.1581209097-1873828646.1581209097


The Doc2Vec method, a concept that was presented in 2014 by Mikilov and Le, the underlying intuition of Doc2Vec is that the document representation should be good enough to predict the words in the document:
https://cs.stanford.edu/~quocle/paragraph_vector.pdf

scikit-learn is a Python module for machine learning:
https://github.com/scikit-learn/scikit-learn

XSS Machine Learning:
https://github.com/Xyntax/ML/blob/master/xss_word_vector/xss_wordvector.ipynb


![Screen%20Shot%202019-04-17%20at%206.08.30%20PM.png](attachment:Screen%20Shot%202019-04-17%20at%206.08.30%20PM.png)


 ### Goal:
    Create a training algorithm for detecting an XSS attack vector in a Query Parameter string.
    Define a set of features that we can use to train the model and label a sample as XSS(1) or Not XSS(0)
    Use a doc2vec vector to determine a vector for sample
    Use additional feature sets to improve the accuracy

## 1. Gather Test Data
Get a list of samples and create an array with labels.

In [ ]:
import sys
!{sys.executable} -m pip install -U numpy
!{sys.executable} -m pip install -U gensim
!{sys.executable} -m pip install -U python-Levenshtein
!{sys.executable} -m pip install -U nltk
!{sys.executable} -m pip install -U scikit-learn
#Import all the dependencies
import warnings
#warnings.filterwarnings("ignore")
import nltk
nltk.download('punkt')

from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
from numpy import *
from urllib.parse import unquote

import numpy as np
import pandas as pd
import csv
import urllib.parse as parse
import pickle


testXSS = []
testNORM = []
X_temp = []
X = []
y = []
xssnum = 0
notxssnum = 0

print("Gathering Data...")
# gather the XSS string and append the label of 1 to y array
with open('lib/testXSS.txt', 'r',encoding='utf-8') as f:
    testXSS = f.readlines()
print("*", sep=' ', end='', flush=True)
# parse out the query part of the URL
for line in testXSS:
    query = parse.urlsplit(line)[3]
    #try to remove open redirect vulns
    if "?http" in str(line):
        continue
    if "?url=http" in str(line):
        continue
    if "?fwd=http" in str(line):
        continue
    if "?path=http" in str(line):
        continue
    if "=http" in str(query):
        continue
    if "page=search" in str(query):
        continue
    if len(query) > 8:
        xssnum += 1
        #X_temp.append(query)
        X_temp.append(line)

# remove duplicates
dedup = list(dict.fromkeys(X_temp))
print("*", sep=' ', end='', flush=True)
# Add a feature to X and label to the y array
for line in dedup:
    #print("XSS => "+line)
    X.append(line)
    y.append(1)

X_temp = []
dedup = []
print("*", sep=' ', end='', flush=True)

# gather the list of normal string and append the label of 0 to y array
with open('lib/testNORM.txt', 'r',encoding='utf-8') as f:
    testNORM = f.readlines()

# parse out the query part of the URL
for line in testNORM:
    query = parse.urlsplit(line)[3]
    #if "http" in str(query):
    #    continue
    if len(query) > 3:
        notxssnum += 1
        X_temp.append(line)

# remove duplicates
dedup = list(dict.fromkeys(X_temp))
print("*", sep=' ', end='', flush=True)
# Add a feature to X and a label to the y array
for line in dedup:
    #print("NOT XSS => "+line)
    X.append(line)
    y.append(0)


#print(X)
#vec2d = [X,y]
#print(vec2d)

'c:\Program' is not recognized as an internal or external command,
operable program or batch file.
'c:\Program' is not recognized as an internal or external command,
operable program or batch file.
'c:\Program' is not recognized as an internal or external command,
operable program or batch file.
'c:\Program' is not recognized as an internal or external command,
operable program or batch file.


'c:\Program' is not recognized as an internal or external command,
operable program or batch file.
[nltk_data] Downloading package punkt to C:\Users\Rabin
[nltk_data]     Subedi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
C:\Users\Rabin Subedi\AppData\Local\Temp\ipykernel_14204\1272350394.py:19: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


Gathering Data...
****

In [ ]:
print("Number of XSS Samples: " + str(xssnum))
print("Number of NOT XSS Samples: " + str(notxssnum))
print("Total Samples: " + str(xssnum + notxssnum))

Number of XSS Samples: 38608
Number of NOT XSS Samples: 44826
Total Samples: 83434


##### The sample set collected was larger then the one used in the reasearch paper
```
We constructed a training dataset containing 1,000 webpages (400 malicious and 600 benign) and their extracted features.

- XSSClassifier: An Efficient XSS Attack Detection Approach Based on Machine Learning Classifier on SNSs
Shailendra Rathore*, Pradip Kumar Sharma*, and Jong Hyuk Park*
```
We could expand the data set and samples used to train the model.

Other resources for collecting sample data are listed below:

Benign:

https://raw.githubusercontent.com/Xyntax/ML/master/DL_for_xss/data/normal_examples.csv
https://raw.githubusercontent.com/Xyntax/ML/master/DL_for_xss/data/white.csv

Malicious:

https://gist.github.com/ThomasOrlita/e2e4a6d72877c8c897082eefe969578a
https://raw.githubusercontent.com/Xyntax/ML/master/DL_for_xss/data/xssed.csv
https://raw.githubusercontent.com/Xyntax/ML/master/DL_for_xss/data/black.csv
https://raw.githubusercontent.com/foospidy/payloads/master/other/xss/reddit_xss_get.txt
https://github.com/danielmiessler/SecLists/tree/master/Fuzzing/XSS


## 2. Create an array of the features for each test sample

Certain HTML tags can be used by an attacker to inject the XSS code scripts from outside. These HTML tags consist of ```<link>, <object>, <form>, <script>, <embed>, <ilayer>, <layer>, <style>, <applet>, <meta>, <img>, <iframe>```.

On the other hand, JavaScript language is used in a webpage for embedding tasks, but an attacker can misuse some methods on the embedded XSS payload such as ```exec(), fromCharCode(), eval, alert(), getElementsByTagName(), write(), unscape(), and escape()```



In [ ]:
# Create a function to convert an array of query strings to a set of features
def getVec(text):
    tagged_data = [
        TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)])
        for i, _d in enumerate(text)
    ]
    max_epochs = 25
    vec_size = 20
    alpha = 0.025

    model = Doc2Vec(
        vector_size=vec_size, alpha=alpha, min_alpha=0.00025, min_count=1, dm=1
    )
    model.build_vocab(tagged_data)
    print("Building the sample vector model...")
    features = []
    for epoch in range(max_epochs):
        # print('Doc2Vec Iteration {0}'.format(epoch))
        print("*", sep=" ", end="", flush=True)
        model.random.seed(42)
        model.train(tagged_data, total_examples=model.corpus_count,
                    epochs=model.epochs)
        # decrease the learning rate
        model.alpha -= 0.0002
        # fix the learning rate, no decay
        model.min_alpha = model.alpha
    model.save("d2v.model")
    print()
    print("Model Saved")
    for i, line in enumerate(text):
        featureVec = [model.dv[i]]
        lineDecode = unquote(line)
        lineDecode = lineDecode.replace(" ", "")
        lowerStr = str(lineDecode).lower()
        # print("X"+str(i)+"=> "+line)
        # We could expand the features
        # https://websitesetup.org/javascript-cheat-sheet/
        # https://owasp.org/www-community/xss-filter-evasion-cheatsheet
        # https://html5sec.org/

        # add feature for malicious HTML tag count
        feature1 = int(lowerStr.count("<link"))
        feature1 += int(lowerStr.count("<object"))
        feature1 += int(lowerStr.count("<form"))
        feature1 += int(lowerStr.count("<embed"))
        feature1 += int(lowerStr.count("<ilayer"))
        feature1 += int(lowerStr.count("<layer"))
        feature1 += int(lowerStr.count("<style"))
        feature1 += int(lowerStr.count("<applet"))
        feature1 += int(lowerStr.count("<meta"))
        feature1 += int(lowerStr.count("<img"))
        feature1 += int(lowerStr.count("<iframe"))
        feature1 += int(lowerStr.count("<input"))
        feature1 += int(lowerStr.count("<body"))
        feature1 += int(lowerStr.count("<video"))
        feature1 += int(lowerStr.count("<button"))
        feature1 += int(lowerStr.count("<math"))
        feature1 += int(lowerStr.count("<picture"))
        feature1 += int(lowerStr.count("<map"))
        feature1 += int(lowerStr.count("<svg"))
        feature1 += int(lowerStr.count("<div"))
        feature1 += int(lowerStr.count("<a"))
        feature1 += int(lowerStr.count("<details"))
        feature1 += int(lowerStr.count("<frameset"))
        feature1 += int(lowerStr.count("<table"))
        feature1 += int(lowerStr.count("<comment"))
        feature1 += int(lowerStr.count("<base"))
        feature1 += int(lowerStr.count("<image"))
        # add feature for malicious method/event count
        feature2 = int(lowerStr.count("exec"))
        feature2 += int(lowerStr.count("fromcharcode"))
        feature2 += int(lowerStr.count("eval"))
        feature2 += int(lowerStr.count("alert"))
        feature2 += int(lowerStr.count("getelementsbytagname"))
        feature2 += int(lowerStr.count("write"))
        feature2 += int(lowerStr.count("unescape"))
        feature2 += int(lowerStr.count("escape"))
        feature2 += int(lowerStr.count("prompt"))
        feature2 += int(lowerStr.count("onload"))
        feature2 += int(lowerStr.count("onclick"))
        feature2 += int(lowerStr.count("onerror"))
        feature2 += int(lowerStr.count("onpage"))
        feature2 += int(lowerStr.count("confirm"))
        feature2 += int(lowerStr.count("marquee"))
        # add feature for ".js" count
        feature3 = int(lowerStr.count(".js"))
        # add feature for "javascript" count
        feature4 = int(lowerStr.count("javascript"))
        # add feature for length of the string
        feature5 = int(len(lowerStr))
        # add feature for "<script"  count
        feature6 = int(lowerStr.count("<script"))
        feature6 += int(lowerStr.count("&lt;script"))
        feature6 += int(lowerStr.count("%3cscript"))
        feature6 += int(lowerStr.count("%3c%73%63%72%69%70%74"))
        # add feature for special character count
        feature7 = int(lowerStr.count("&"))
        feature7 += int(lowerStr.count("<"))
        feature7 += int(lowerStr.count(">"))
        feature7 += int(lowerStr.count('"'))
        feature7 += int(lowerStr.count("'"))
        feature7 += int(lowerStr.count("/"))
        feature7 += int(lowerStr.count("%"))
        feature7 += int(lowerStr.count("*"))
        feature7 += int(lowerStr.count(";"))
        feature7 += int(lowerStr.count("+"))
        feature7 += int(lowerStr.count("="))
        feature7 += int(lowerStr.count("%3C"))
        # add feature for http count
        feature8 = int(lowerStr.count("http"))

        # append the features
        featureVec = np.append(featureVec, feature1)
        # featureVec = np.append(featureVec,feature2)
        featureVec = np.append(featureVec, feature3)
        featureVec = np.append(featureVec, feature4)
        featureVec = np.append(featureVec, feature5)
        featureVec = np.append(featureVec, feature6)
        featureVec = np.append(featureVec, feature7)
        # featureVec = np.append(featureVec,feature8)
        # print(featureVec)
        features.append(featureVec)
    return features

#### Using the function getVec(['data'])
```
testText = [
                'Search=%22%3E\'%3E%3CSCRIPT%20SRC=http://br.zone-h.org/testes/xss.js%3E%3C/SCRIPT%3E?',
                'id=15%3Cscript%3Ealert%28document.cookie%29%3C/script%3E',
                'composite_search=1&keyword="/><script>alert("Xss:Vijayendra")</script>'
            ]

Xnew = getVec(testText)
```

In [ ]:
features = getVec(X)
features_dict = {"data": X, "features": features, "label": y}

Building the sample vector model...
*************************
Model Saved


### Features Data

In [ ]:
print("Test Sample: " + X[0])
print("Features: " + str(features[0]))
print(
    "\nLabel:\033[1;31;1m XSS(1)/\033[1;32;1m NOT XSS(0)\033[0;0m: " + str(y[0]))

Test Sample: http://search.rin.ru/cgi-bin/find.cgi?text=%3Cscript%3Ealert(%27HZ+iz+1337%27)%3B%3C%2Fscript%3E

Features: [ 2.95006490e+00 -2.74251771e+00  4.69490469e-01  1.08792460e+00
  1.16353869e+00 -1.64095819e+00  1.41985941e+00 -2.07024646e+00
  6.59130216e-01  2.67131639e+00  2.42388821e+00  3.27396083e+00
  7.06738949e-01  3.58186871e-01 -3.89876664e-01 -2.51898980e+00
 -8.89518738e-01  8.75902772e-01 -2.63400030e+00 -3.78373191e-02
  0.00000000e+00  0.00000000e+00  0.00000000e+00  8.10000000e+01
  1.00000000e+00  1.50000000e+01]

Label: XSS(1)/ NOT XSS(0): 1


## 3. Train the Model

Train/Test split the training data

In [ ]:
from sklearn.model_selection import train_test_split


np.random.seed(42)


X_train, X_test, y_train, y_test = train_test_split(
    features, y, test_size=0.3, random_state=42
)

In [1]:

from sklearn.neural_network import MLPClassifier
my_classifier6 = MLPClassifier(max_iter=2000, random_state=42)
print(my_classifier6)
print()

MLPClassifier(max_iter=2000, random_state=42)



#### Additonal tunning of hyperparameters can be done.

I have only set the KNN n_neighbors weights

In [ ]:
# print("Training Classifier #1 DecisionTreeClassifier")
# my_classifier1.fit(X_train, y_train)
# print("Training Classifier #2 SVC")
# my_classifier2.fit(X_train, y_train)
# print("Training Classifier #3 GaussianNB")
# my_classifier3.fit(X_train, y_train)
# print("Training Classifier #4 KNeighborsClassifier")
# my_classifier4.fit(X_train, y_train)
# print("Training Classifier #5 RandomForestClassifier")
# my_classifier5.fit(X_train, y_train)
print("Training Classifier #6 MLPClassifier")
my_classifier6.fit(X_train, y_train)

# predictions1 = my_classifier1.predict(X_test)
# predictions2 = my_classifier2.predict(X_test)
# predictions3 = my_classifier3.predict(X_test)
# predictions4 = my_classifier4.predict(X_test)
# predictions5 = my_classifier5.predict(X_test)
predictions6 = my_classifier6.predict(X_test)

Training Classifier #6 MLPClassifier


## 4. Evaluation Metrics


### Predictions Results

Test the classifiers and compare the results of the models used.

### Training Accuracy Score

In [ ]:
from sklearn.metrics import accuracy_score


print("Accuracy Score #6: {:.1%}".format(accuracy_score(y_test, predictions6)))

Accuracy Score #6: 99.6%


##### How did the test compare with the research paper

Comparing the data with the results seen in the research paper we are within the expected accuracy range.
```
Every classifier had recall, precision of more than 0.950, F-measure of over 0.952, accuracy of more than 0.947, and false positive rate of below 2.24. These results validate the effectiveness of our proposed approach. The overall result of all classifiers shows that the tree classifiers (RandomForest and ADTree) obtained the best performance compared to meta-classifiers (Decorate and AdaBoost.M1). RandomForest is the best classifier with recall of 0.971, precision of 0.977, F- measure of 0.974, accuracy of 0.972, and lowest false positive rate of 0.87.

- XSSClassifier: An Efficient XSS Attack Detection Approach Based on Machine Learning Classifier on SNSs
Shailendra Rathore*, Pradip Kumar Sharma*, and Jong Hyuk Park*
```

### Classification Report

In [ ]:
from sklearn.metrics import classification_report

print("Classification Report #6 MLPClassifier")

print(classification_report(y_test, predictions6))

Classification Report #6 MLPClassifier
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     13437
           1       0.99      1.00      1.00     11594

    accuracy                           1.00     25031
   macro avg       1.00      1.00      1.00     25031
weighted avg       1.00      1.00      1.00     25031



### Confusion Matrix
![cm.png](attachment:cm.png)


In [ ]:
from sklearn.metrics import confusion_matrix

print("\nConfusion Matrix #6 MLPClassifier")


print(confusion_matrix(y_test, predictions6))


Confusion Matrix #6 MLPClassifier
[[13377    60]
 [   45 11549]]


## 5 . Retrain the Model with All the Data
Retrain the model with all the data gathered and then run some test with different input that is not included in the testing data

https://machinelearningmastery.com/make-predictions-scikit-learn/



In [ ]:

print("Training Classifier #6 MLPClassifier")
my_classifier6.fit(features, y)

Training Classifier #6 MLPClassifier


MLPClassifier(max_iter=2000, random_state=42)

In [ ]:
# save the model to disk
# filename1 = "lib/DecisionTreeClassifier.sav"
# pickle.dump(my_classifier1, open(filename1, "wb"))

# filename2 = "lib/SVC.sav"
# pickle.dump(my_classifier2, open(filename2, "wb"))

# filename3 = "lib/GaussianNB.sav"
# pickle.dump(my_classifier3, open(filename3, "wb"))

# filename4 = "lib/KNeighborsClassifier.sav"
# pickle.dump(my_classifier4, open(filename4, "wb"))

# filename5 = "lib/RandomForestClassifier.sav"
# pickle.dump(my_classifier5, open(filename5, "wb"))

filename6 = "MLPClassifier.sav"
pickle.dump(my_classifier6, open(filename6, "wb"))

In [ ]:
# load the model from disk
# loaded_model1 = pickle.load(open(filename1, "rb"))
# loaded_model2 = pickle.load(open(filename2, "rb"))
# loaded_model3 = pickle.load(open(filename3, "rb"))
# loaded_model4 = pickle.load(open(filename4, "rb"))
# loaded_model5 = pickle.load(open(filename5, "rb"))
loaded_model6 = pickle.load(open(filename6, "rb"))

### Test the Model with Data it has not seen yet


https://www.netnea.com/cms/2016/04/12/xss-payload-from-reddit-used-against-owasp-modsecurity-core-rules/

https://raw.githubusercontent.com/foospidy/payloads/master/other/xss/reddit_xss_get.txt

https://github.com/foospidy/payloads/tree/master/other/xss


In [ ]:
testXSS = [
    "<script>alert('xss')</script><script><script>",
    "hellomo",
    "https://store.bentley.com/en/shop/search?term=%22%3E%3Cdetails%20open%20ontoggle=prompt(1337)%3ExxLouisLouisLouis",
    "ghfdhgdhjgd",
    "uid%3D19%26list_page%3D%22%3E%3Cscript%3Ealert%28document.cookie%29%3B%3C/script%3E",
    "&template=en_search_error&postalCode=\\';alert(0)//",
    "&where=%3Cscript%3Ealert%28%27xss%27%29%3C%2Fscript%3E&loctypes=1003%2C1001%2C1000%2C1%2C9%2C5%2C11%2C13%2C19%2C20&from=hdr_localsearch",
    "http://mydata.com/sad/sd/qwd/qwde/qwe/?sessionid=12",
    "http://mydata.com?id=script",
    "&';}},{scope:'email,user_about_me,user_hometown,user_interests,user_likes,user_status,user_website,user_birthday,publish_stream,publish_actions,offline_access'});}alert(0);b=function(response){c=({a:{//",
    "http://myurl.com?<script",
    "http://mydata.com?script=script",
    'composite_search=1&keyword="/><script>alert("Xss:Vijayendra")</script>',
    "http://mysite.com?srtalert",
    "script",
    "alert",
    "Search=%22%3E'%3E%3CSCRIPT%20SRC=http://br.zone-h.org/testes/xss.js%3E%3C/SCRIPT%3E?",
    "id=15%3Cscript%3Ealert%28document.cookie%29%3C/script%3E",
    'composite_search=1&keyword="/><script>alert("Xss:Vijayendra")</script>',
    "id=123&href=abdc<a<script>alert(1)",
    "<<<<<<>>>>></>,><><>",
    "alert()alert()",
    "alertalert",
    "?url=http://localhost:8888/notebooks/Documents/MachineLearning/Practical%20Machine%20Learning",
    "<script<script",
    "<scriptalert",
    "httphttphttp",
    "https://disqus.com/?ref_noscript",
    "I am a string",
    '<img src="javascript:alert(1)/>"',
    "HelloWorld!",
    "http://mysite.com?<script>",
    "<input type=\"text\" value=`` <div/onmouseover='alert(471)'>X</div>",
    '<img \x47src=x onerror="javascript:alert(324)">',
    '<a href="\xE2\x80\x87javascript:javascript:alert(183)" id="fuzzelement1">test</a>',
    "<body onscroll=javascript:alert(288)><br><br><br><br><br><br>...<br><br><br><br><br><br><br><br><br><br>...<br><br><br><br><br><br><br><br><br><br>...<br><br><br><br><br><br><br><br><br><br>...<br><br><br><br><br><br><br><br><br><br>...<br><br><br><br><input autofocus>",
    '<meta charset="mac-farsi">¼script¾javascript:alert(379)¼/script¾',
    '<HTML xmlns:xss><?import namespace=(493)s" implementation="%(htc)s"><xss:xss>XSS</xss:xss></HTML>""","XML namespace."),("""<XML ID=(494)s"><I><B>&lt;IMG SRC="javas<!-- -->cript:javascript:alert(420)"&gt;</B></I></XML><SPAN DATASRC="#xss" DATAFLD="B" DATAFORMATAS="HTML"></SPAN>',
]


# print(Xnew)

In [ ]:
Xnew = getVec(testXSS)
ynew6 = loaded_model6.predict(Xnew)

Building the sample vector model...
****

*********************
Model Saved


##### Display the results

In [ ]:
# show the sample inputs and predicted outputs using a wighted value
xssCount = 0
notXssCount = 0
for i in range(len(Xnew)):
    print(ynew6[i])
    # print(score)
    if ynew6[i] >= 0.5:
        print("\033[1;31;1mXSS\033[0;0m => " + testXSS[i])
        xssCount += 1
    else:
        print("\033[1;32;1mNOT XSS\033[0;0m => " + testXSS[i])
        notXssCount += 1

print()
print("*------------- RESULTS -------------*")
print("\033[1;31;1mXSS\033[0;0m => " + str(xssCount))
print("\033[1;32;1mNOT XSS\033[0;0m => " + str(notXssCount))

1
XSS => <script>alert('xss')</script><script><script>
0
NOT XSS => hellomo
0
NOT XSS => https://store.bentley.com/en/shop/search?term=%22%3E%3Cdetails%20open%20ontoggle=prompt(1337)%3ExxLouisLouisLouis
0
NOT XSS => ghfdhgdhjgd
1
XSS => uid%3D19%26list_page%3D%22%3E%3Cscript%3Ealert%28document.cookie%29%3B%3C/script%3E
0
NOT XSS => &template=en_search_error&postalCode=\';alert(0)//
1
XSS => &where=%3Cscript%3Ealert%28%27xss%27%29%3C%2Fscript%3E&loctypes=1003%2C1001%2C1000%2C1%2C9%2C5%2C11%2C13%2C19%2C20&from=hdr_localsearch
0
NOT XSS => http://mydata.com/sad/sd/qwd/qwde/qwe/?sessionid=12
0
NOT XSS => http://mydata.com?id=script
1
XSS => &';}},{scope:'email,user_about_me,user_hometown,user_interests,user_likes,user_status,user_website,user_birthday,publish_stream,publish_actions,offline_access'});}alert(0);b=function(response){c=({a:{//
1
XSS => http://myurl.com?<script
0
NOT XSS => http://mydata.com?script=script
1
XSS => composite_search=1&keyword="/><script>alert("Xss:Vijayendra")</s

In [ ]:
print(ynew6)

[1 0 0 0 1 0 1 0 0 1 1 0 1 0 0 0 1 1 1 1 1 0 0 0 1 1 0 0 0 1 0 1 1 1 1 1 1
 1]


In [ ]:
if '1' in ynew6:
    print('yes')

yes
